In [1]:
import math
import numpy as np
import tensorflow as tf

In [2]:
def get_y(x):
    return 10 + x*x

def sample_data(n=10000, scale=100):
    data = []

    x = scale*(np.random.random_sample((n,))-0.5)

    for i in range(n):
        yi = get_y(x[i])
        data.append([x[i], yi])

    return np.array(data)


In [48]:
def generator(hsize=[16, 16]):

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(units=hsize[0], activation=tf.nn.leaky_relu),
            tf.keras.layers.Dense(units=hsize[1], activation=tf.nn.leaky_relu),
            tf.keras.layers.Dense(units=2),
        ]
    )

    return model
Z = tf.keras.Input(dtype=tf.float32, shape=[None, 1])
G_sample = generator()
G_sample.compile(optimizer='rmsprop',
                           loss = tf.keras.losses.BinaryCrossentropy(),
                           metrics=[tf.keras.metrics.BinaryAccuracy(),
                                    tf.keras.metrics.FalseNegatives()])

G_sample.build(input_shape=[None, 1])
G_sample.summary()


Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_134 (Dense)           (None, 16)                32        
                                                                 
 dense_135 (Dense)           (None, 16)                272       
                                                                 
 dense_136 (Dense)           (None, 2)                 34        
                                                                 
Total params: 338
Trainable params: 338
Non-trainable params: 0
_________________________________________________________________


In [44]:
def discriminator(hsize=[16, 16]):

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(units=hsize[0], activation=tf.nn.leaky_relu),
            tf.keras.layers.Dense(units=hsize[1], activation=tf.nn.leaky_relu),
            tf.keras.layers.Dense(units=2),
            tf.keras.layers.Dense(units=1) 
        ]
    )

    return model

X = tf.keras.Input(dtype=tf.float32, shape=[None, 2])
discriminator_fake = discriminator()

discriminator_real = discriminator()
print(discriminator_fake)
print(discriminator_real)

discriminator_real.compile(optimizer='rmsprop',
                           loss = tf.keras.losses.CategoricalCrossentropy(),
                           metrics=[tf.keras.metrics.BinaryAccuracy(),
                                    tf.keras.metrics.FalseNegatives()])

discriminator_real.build(input_shape=[None, 2])

discriminator_real.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_130 (Dense)           (None, 16)                48        
                                                                 
 dense_131 (Dense)           (None, 16)                272       
                                                                 
 dense_132 (Dense)           (None, 2)                 34        
                                                                 
 dense_133 (Dense)           (None, 1)                 3         
                                                                 
Total params: 357
Trainable params: 357
Non-trainable params: 0
_________________________________________________________________


In [45]:
disc_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits, labels=tf.ones_like(r_logits)) 
    + tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.zeros_like(f_logits))
    )
gen_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.ones_like(f_logits))
    )


In [47]:
for i in range(10):
    data = sample_data()
    print(data.shape)
    G_sample.train_on_batch(
    x = data[:, 0:1],
    y=data[:, 1])

(10000, 2)


TypeError: 'NoneType' object is not callable

In [ ]:
gen_vars = tf.Graph(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
disc_vars = tf.Graph(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Discriminator")

gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_loss,var_list = gen_vars) # G Train step
disc_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(disc_loss,var_list = disc_vars) # D Train step


In [ ]:
for i in range(100001):
    X_batch = sample_data(n=batch_size)
    Z_batch = sample_Z(batch_size, 2)
    _, dloss = sess.run([disc_step, disc_loss], feed_dict={X: X_batch, Z: Z_batch})
    _, gloss = sess.run([gen_step, gen_loss], feed_dict={Z: Z_batch})

    print "Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i,dloss,gloss)

In [ ]:
tf.nn.leaky_relu